In this notebook, we make a CNN model with 3 conv2D layer, 1 flatten and 1 dense layer. We turn of training in the very beginning and train + evaluate on the training and test dataset. After that, we evaluate 10 most irrelevant filters based upon their average sum of their activations on 3 different axes (0,1,2). '0' takes all the datasets, '1' is along the height, and '2' is along the width. After removing the irrelevant filters, we train+evaluate to check the accuracy and time elapsed on training.

In [6]:
#defining imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras import datasets
from keras.layers import Dense
from keras.layers import Conv2D, Layer
import tensorflow_model_optimization as tfmot

In [7]:
# get MNIST fashion
from keras.datasets import fashion_mnist
(fashion_train_img, fashion_train_labels), (fashion_test_img, fashion_test_labels) = fashion_mnist.load_data()

In [8]:
#adding 4th dimension as 1 to declare as grayscale image
#normalizing the images
fashion_train_img = fashion_train_img.reshape((60000, 28, 28, 1))
fashion_train_img = fashion_train_img/255.0

fashion_test_img = fashion_test_img.reshape((10000, 28, 28, 1))
fashion_test_img = fashion_test_img/255.0

Defining our model

In [9]:
# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv2d_1'),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_2'),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_3'),
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])

Turning off Backprop for Conv2D layers

In [10]:
for layer in model.layers[:3]:
    layer.trainable=False
    #print (layer)


In [11]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train the model and display the activations after each epoch
start_time = time.time()
history = model.fit(fashion_train_img, fashion_train_labels, epochs=1, validation_data=(fashion_test_img, fashion_test_labels))
end_time = time.time()

   1/1875 [..............................] - ETA: 5:46 - loss: 2.3008 - accuracy: 0.0938

2023-06-23 11:39:01.390104: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 11s 6ms/step - loss: 0.5746 - accuracy: 0.8081 - val_loss: 0.4752 - val_accuracy: 0.8353


In [13]:
test_loss, test_acc = model.evaluate(fashion_test_img, fashion_test_labels, verbose=2)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

313/313 - 1s - loss: 0.4752 - accuracy: 0.8353 - 1s/epoch - 4ms/step
Test accuracy: 0.8353000283241272
Time elapsed:  10.80650806427002


Filter Pruning for 1st Convolutional Layers

In [14]:
# Get the activations of the convolutional layer
layer_name = 'conv2d_1'
activation_model = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
activations = activation_model.predict(fashion_train_img)

# Determine the filter indices with the least activations
filter_activations = np.sum(activations, axis=(0, 1, 2))
indices_to_prune = np.argsort(filter_activations)[:15]  # Prune the 10 filters with the least activations

# Convert the weights of the pruned filters to zero
weights, biases = model.get_layer(layer_name).get_weights()
weights[:, :, :, indices_to_prune] = 0
model.get_layer(layer_name).set_weights([weights, biases])

1875/1875 [==============================] - 2s 1ms/step


Filter Pruning for 2nd Convolutional Layers

In [15]:
# Get the activations of the convolutional layer
layer_name = 'conv2d_2'
activation_model = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
activations = activation_model.predict(fashion_train_img)

# Determine the filter indices with the least activations
filter_activations = np.sum(activations, axis=(0, 1, 2))
indices_to_prune = np.argsort(filter_activations)[:15]  # Prune the 10 filters with the least activations

# Convert the weights of the pruned filters to zero
weights, biases = model.get_layer(layer_name).get_weights()
weights[:, :, :, indices_to_prune] = 0
model.get_layer(layer_name).set_weights([weights, biases])

1875/1875 [==============================] - 5s 3ms/step


Filter Pruning for 3rd Convolutional Layers

In [16]:
# Get the activations of the convolutional layer
layer_name = 'conv2d_3'
activation_model = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
activations = activation_model.predict(fashion_train_img)

# Determine the filter indices with the least activations
filter_activations = np.sum(activations, axis=(0, 1, 2))
indices_to_prune = np.argsort(filter_activations)[:15]  # Prune the 10 filters with the least activations

# Convert the weights of the pruned filters to zero
weights, biases = model.get_layer(layer_name).get_weights()
weights[:, :, :, indices_to_prune] = 0
model.get_layer(layer_name).set_weights([weights, biases])

1875/1875 [==============================] - 6s 3ms/step


In [17]:
# Train the model and display the activations after each epoch
start_time = time.time()
history = model.fit(fashion_train_img, fashion_train_labels, epochs=1, validation_data=(fashion_test_img, fashion_test_labels))
end_time = time.time()

1875/1875 [==============================] - 10s 5ms/step - loss: 0.4835 - accuracy: 0.8357 - val_loss: 0.4794 - val_accuracy: 0.8294


In [18]:
# Evaluate the pruned model
_, pruned_accuracy = model.evaluate(fashion_train_img, fashion_train_labels, verbose=0)
print('Pruned accuracy:', pruned_accuracy)
print('Time elapsed: ', end_time - start_time)


Pruned accuracy: 0.8446666598320007
Time elapsed:  9.936978816986084
